In [ ]:
from context import *
from stable_baselines3 import PPO,A2C,SAC,TD3,DQN,DDPG
from stable_baselines3.common.save_util import load_from_zip_file
from stable_baselines3.common.monitor import Monitor
import torch as th
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

from pprint import pprint
import enum

import rlrom.wrappers.stl_wrapper
import stlrom
from rlrom.envs import *
import rlrom.utils
import time
import matplotlib.pyplot as plt


In [ ]:
class EnvMode(enum.Enum):
    VANILLA=0
    TERM_SLOW=1
    
def make_env(train=True, env_mode=EnvMode.TERM_SLOW, verbose=0):
    if train:
        env = gym.make("highway-fast-v0")
    else:
        env = gym.make("highway-v0", render_mode='human')

    env.unwrapped.configure({
            "observation": {"type": "Kinematics"},
                "action": {
                    "type": "DiscreteMetaAction",
                },
                "lanes_count": 4,
                "vehicles_count": 50,
                "controlled_vehicles": 1,
                "initial_lane_id": None,
                "duration": 100,  # [s]
                "ego_spacing": 2,
                "vehicles_density": 1,
                "collision_reward": -.4,  # The reward received when colliding with a vehicle.
                "right_lane_reward": 0,  # The reward received when driving on the right-most lanes, linearly mapped to
                # zero for other lanes.
                "high_speed_reward": 1.,  # The reward received when driving at full speed, linearly mapped to zero for
                # lower speeds according to config["reward_speed_range"].
                "lane_change_reward": 0,  # The reward received at each lane change action.
                "reward_speed_range": [20, 30],
                "normalize_reward": True,
                "offroad_terminal": False,        
    })

    if env_mode==EnvMode.TERM_SLOW:
        cfg = cfg_envs['highway-env']
        driver= stlrom.STLDriver()
        driver.parse_string(cfg['specs'])        
        env = rlrom.wrappers.stl_wrapper.STLWrapper(env,driver,signals_map=cfg, terminal_formulas={'ego_slow_too_long'})

    if verbose>=1:
        pprint(cfg)
    return env

# Training vanilla highway-env

In [ ]:
n_cpu = 12
batch_size = 64
neurons = 128
policy_kwargs = dict(
    #activation_fn=th.nn.ReLU,
    net_arch=dict(pi=[neurons, neurons], qf=[neurons, neurons])
)

vec_env = make_vec_env(make_env, n_envs=n_cpu, vec_env_cls=SubprocVecEnv)
model = PPO(
     "MlpPolicy",
     vec_env,
     device='cpu',
     policy_kwargs=policy_kwargs,
     n_steps=batch_size * 12 // n_cpu,
     batch_size=batch_size,
     n_epochs=10,
     learning_rate=7e-4,
     gamma=0.9,
     verbose=1,
     tensorboard_log="./highway_ppo/"
)

In [ ]:
# Train the agent
model.learn(
    total_timesteps=100_000,
    progress_bar=True
)


In [ ]:

model.save('ppo_model')

# Testing

In [ ]:
model = PPO.load('ppo_model.zip')

In [ ]:
env_mode = EnvMode.TERM_SLOW
env = make_env(train=False,env_mode=EnvMode.TERM_SLOW, verbose=0)
env.unwrapped.configure({
            "observation": {"type": "Kinematics"},
                "action": {
                    "type": "DiscreteMetaAction",
                },
                "lanes_count": 4,
                "vehicles_count": 50,
                "controlled_vehicles": 1,
                "initial_lane_id": None,
                "duration": 100,  # [s]
                "ego_spacing": 2,
                "vehicles_density": 1,
                "collision_reward": -.1,  # The reward received when colliding with a vehicle.
                "right_lane_reward": 0,  # The reward received when driving on the right-most lanes, linearly mapped to
                # zero for other lanes.
                "high_speed_reward": 0.,  # The reward received when driving at full speed, linearly mapped to zero for
                # lower speeds according to config["reward_speed_range"].
                "lane_change_reward": 0.,  # The reward received at each lane change action.
                "reward_speed_range": [20, 30],
                "normalize_reward": False,
                "offroad_terminal": False,
                "manual_control": True        
    })

obs, info = env.reset()
env.stl_driver.set_param('v_slow', 0.3)
env.stl_driver.set_param('v_fast', 0.35)
#wobs = env.wrapped_obs
for _ in range(100):    
    #action, _states = model.predict(wobs, deterministic=True)
    action, _states = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)    
    wobs= env.wrapped_obs

    if terminated:
        print('Crash')
        break    
env.close()

lay = """
 ego_x
 ego_slow
 ego_slow_too_long
"""
lay = utils.get_layout_from_string(lay)

width = 12
height = 4
fig, axs = plt.subplots(len(lay),1, figsize=(width, height))

idx_ax =0
for sig_list in lay:
    for sig in sig_list:
        if len(lay)>1:
            env.plot_signal(sig, axs[idx_ax])
        else:
            env.plot_signal(sig, axs)
    idx_ax +=1

In [ ]:
print(model.observation_space)

In [ ]:
env.close()

In [ ]:

fig, ax = plt.subplots()
env.plot_signal("reward", ax)
# Show the plot with interactive features
plt.show()


In [ ]:
def get_fig(env, signals_layout):
    lay = utils.get_layout_from_string(signals_layout)
    status = "Plot ok. Hit reset on top right if not visible."            
    #f= figure(height=200)
    figs = []
    colors = itertools.cycle(palette)    
    for signal_list in enumerate(lay):
        f=None
        for signal in signal_list[1]:                
            #try: 
                color=colors.__next__()                    
                #tr_idx = self.trace_idx
                print(signal.strip())
                if signal.strip().startswith("set_trace_idx(") or signal.strip().startswith("_tr("):            
                    tr_idx = int(signal.split('(')[1][:-1])                         
                    env.set_current_trace(tr_idx)                        
                else: 
                    if f is None:
                        if figs == []:
                            f = figure(height=200)
                        else:
                            f = figure(height=200, x_range=figs[0][0].x_range)
                        figs.append([f])
                    env.plot_signal(f, signal, color=color)
            #except:
            #     status = "Warning: error getting values for " + signal
    fig = gridplot(figs, sizing_mode='stretch_width')        
    
    return fig, status
                

In [ ]:
def db_plot_signal(self, signal, fig=None,label=None,  color=None, online=False, horizon=0):
    # signal should be part of the "signal" declaration or a valid formula id 
     
        if self.stl_driver.data == []:
            raise ValueError("No data to plot.")
                 
        time = self.get_time()

        if signal in self.signals_map:
            signal_index = list(self.signals_map.keys()).index(signal)+1        
            sig_values = [s[signal_index] for s in self.stl_driver.data]
            if label is None:
                label=signal
        elif signal in self.formulas:
            sig_values = self.get_rob(signal, online=online,horizon=horizon)
            signal_index = self.formulas.index(signal)+len(self.signals_map)        
            if label is None:
                label=signal
        elif isinstance(signal, np.ndarray) and signal.shape == (len(self.get_time()),):
            sig_values = signal
        elif isinstance(signal, stlrom.Signal):
            pass
        else:
            try:
                sig_values = self.get_rob(signal, online=online,horizon=horizon)
            except Exception as e:
               raise ValueError(f"Name '{signal}' not in signals_map nor in parsed formulas")

        if fig is None:
            fig = figure(height=200)

        fig.xaxis.axis_label = "Time"
        print(sig_values)
        fig.step(time, sig_values)            
        
        
        return fig


In [ ]:
f = figure(height=200)
db_plot_signal(env, "ego_x")


In [ ]:
f.

In [ ]:
fig, status = get_fig(env, lay)